In [237]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML


pd.get_option("display.max_columns")
df2 = pd.read_csv('cup98lrn.txt', sep=',')
df1 = pd.read_csv('cup98lrn.txt', sep=',')


/tmp/ipykernel_762942/2428886817.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('cup98lrn.txt', sep=',')
/tmp/ipykernel_762942/2428886817.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('cup98lrn.txt', sep=',')


In [251]:
'''
Data preprocessing tasks include the following(https://kdd.org/cupfiles/KDDCupData/1998/cup98doc.txt)

1. Noisy Data
Some of the fields in the analysis file may contain data entry and/or
formatting errors. You are expected to clean these fields (without
excluding the records.)
Rui: I think dataframe already takes care of this part. Dont think there is any data record as formatting issue

2. Records and Fields with Missing and Sparse Data
    2.1:Drop the attribute when it missing more than 99.5% 
    2.2:replace the missing record with the (mean,mode,avg, most common type, etc) 
            value of the avg. value of attribute
3.Fields Containing Constants
4.Time Frame and Date Fields     
'''

def trim_data(df):
    
    #I think 1 is already done by dataframe
    
    
    ''' General:
        replacing any value with period or/and whitespace
    '''
    df.replace(r'^\s.*$', np.nan, regex=True, inplace=True)
    
    num_cols = len(df.columns)
    num_rows = len(df)
    
    df.dropna(labels='CONTROLN',axis = 1, inplace=True)
    
    ####TARGET_B#####
    df.dropna(labels='TARGET_B',axis = 1, inplace=True)
    
    
    ####Fields Containing Constants################
    df.dropna(axis = 0, thresh= 2, inplace=True)
    
    ####dealing with missing values#################
        
    #1. drop the attribute if missing values >= 99.5%
    #calculating the dropping_treshold 
    perc = 99.5
    min_count =  int(((100-perc)/100)*num_rows+ 1)
    df.dropna(axis = 1, thresh=min_count, inplace=True)
    
    #2. if features contains NAN < 99.5% we need to replace NAN with the most frequent value
    #this line does replace differnet attribute types(Number, char, boolean, etc)  with the most frequent
    # value
    df.fillna(df.mode().iloc[0], inplace=True)
    
    
    ####dealing with sparse values#################
    
    
    ####Time Frame and Date Fields#########
    
    
    

In [252]:
display(df1)

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,B,P,3712,0,X,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,B,P,5202,0,X,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,B,P,0,0,X,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,B,P,2801,0,X,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,MBC,0,FL,33176,B,P,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,B,P,0,0,X,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,B,P,5001,0,X,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,B,P,3801,0,X,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,B,P,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [255]:
# import re
# line = "something with space . dot"
# line = re.sub(r'[. \n]+', ",", line)
# print(line)

something,with,space,dot


In [259]:
d = {'col1': [1, ' '], 'col2': [' .', 4]}
df = pd.DataFrame(data=d)
df.replace(r'^\s.*$', np.nan, regex=True, inplace=True)
display(df)

,col1,col2
0,1.0,NaN
1,NaN,4.0
